# Binary Segmentation using U-Net

CSDMS	2019	Annual	Meeting:	CSDMS	3.0	– Bridging	Boundaries

May	21-23,	2019

### Daniel Buscombe

Assistant Research Professor

School of Earth and Sustainability

School of Informatics, Computing and Cyber Systems


Northern Arizona University, Flagstaff, AZ

[Email](mailto:daniel.buscombe@nau.edu)
[Web](www.danielbuscombe.com)
[Google Scholar](https://scholar.google.com/citations?user=bwVl0NwAAAAJ&hl=en)


## Introduction

The U-Net model is a simple fully  convolutional neural network that is used for binary segmentation i.e foreground and background pixel-wise classification. Mainly, it consists of two parts. 

*   Encoder: we apply a series of conv layers and downsampling layers  (max-pooling) layers to reduce the spatial size 
*   Decoder: we apply a series of upsampling layers to reconstruct the spatial size of the input. 

The two parts are connected using a concatenation layers among different levels. This allows learning different features at different levels. At the end we have a simple conv 1x1 layer to reduce the number of channels to 1. 


![alt text](https://blog.playment.io/wp-content/uploads/2018/03/Screen-Shot-2018-09-05-at-3.00.03-PM.png)

## Imports

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate, BatchNormalization, UpSampling2D
from keras.layers import  Dropout, Activation
from keras.optimizers import Adam, SGD
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import backend as K
from keras.utils import plot_model
import tensorflow as tf
import random
import cv2
from random import shuffle
from glob import glob
from imageio import imread, imwrite

## Dataset


Download the data set from github

In [0]:
! rm -rf Unet_binary_example

In [0]:
! git clone --depth 1 https://github.com/dbuscombe-usgs/Unet_binary_example.git

Change directory

In [0]:
import os
os.chdir('./Unet_binary_example')
print(os.getcwd())

Note that we have two folders. The first one is `images` which contains the raw images and `annotations` which contains the `labels`imagery, and an empty `masks` folder

In [0]:
files = glob('annotations/labels/*.tif')
len(files)

In [0]:
labs = np.genfromtxt('annotations/class_dict.csv', delimiter=',', skip_header=1, dtype=None)
print(labs)

Next, define the class that we want to treat as the foreground in the binary segmentation

In [0]:
class_foregrnd = 'water' #'veg'

The following code creates binary masks by labeling all water pixels 1, and all others 0, and writes them to the `masks` folder

In [0]:
index = np.where(np.asarray([k[0].decode()==class_foregrnd for k in labs])==True)[0]

In [0]:
for k in range(len(files)):
  i0 = (imread(files[k])[:,:,0]==int(labs[index][0][1])).astype('uint8')
  i1 = (imread(files[k])[:,:,1]==int(labs[index][0][2])).astype('uint8')
  i2 = (imread(files[k])[:,:,2]==int(labs[index][0][3])).astype('uint8')
  mask = ((i0+i1+i2)==3).astype('uint8')
  imwrite(files[k].replace('labels', 'masks'), mask)

Now let's take a look at the contents of the `masks` folder. There should be the same number of images as in the `labels` folder

In [0]:
files = glob('annotations/masks/*.tif')
len(files)

## Setting up model training

The next function creates a generator that will randomly draw images from the training and testing sets

In [0]:
def image_generator(files, batch_size = 32, sz = (512, 512)):
  
  while True: 
    
    #extract a random batch 
    batch = np.random.choice(files, size = batch_size)    
    
    #variables for collecting batches of inputs and outputs 
    batch_x = []
    batch_y = []
    
    
    for f in batch:

        #get the masks. Note that masks are png files 
        mask = Image.open(glob(f'annotations/masks/{f[:-4]}*.tif')[0])
        mask = np.array(mask.resize(sz))


        #preprocess the mask 
        mask[mask >= 2] = 0 
        mask[mask != 0 ] = 1
        
        batch_y.append(mask)

        #preprocess the raw images 
        raw = Image.open(f'images/{f}')
        raw = raw.resize(sz)
        raw = np.array(raw)

        #check the number of channels because some of the images are RGBA or GRAY
        if len(raw.shape) == 2:
          raw = np.stack((raw,)*3, axis=-1)

        else:
          raw = raw[:,:,0:3]

        batch_x.append(raw)

    #preprocess a batch of images and masks 
    batch_x = np.array(batch_x)/255.
    batch_y = np.array(batch_y)
    batch_y = np.expand_dims(batch_y,3)

    yield (batch_x, batch_y)      
    

Next we set our batch size, and the proportion of all images to use for training

In [0]:
batch_size = 3

prop_train = 0.8

all_files = os.listdir('images')
shuffle(all_files)

split = int(prop_train * len(all_files))

#split into training and testing
train_files = all_files[0:split]
test_files  = all_files[split:]

train_generator = image_generator(train_files, batch_size = batch_size)
test_generator  = image_generator(test_files, batch_size = batch_size)

In [0]:
print(len(train_files))
print(len(test_files))

The next few lines randomly select an image and display it, the associated mask, and the masked image. Rerun this cell again and again to see a different image every time

In [0]:
x, y= next(train_generator)
plt.axis('off')
img = x[0]
msk = y[0].squeeze()
msk = np.stack((msk,)*3, axis=-1)
nx, ny,nz = np.shape(msk)

plt.imshow( np.concatenate([img, msk, img*msk], axis = 1))

## IoU metric

The intersection over union (IoU) metric is a simple metric used to evaluate the performance of a segmentation algorithm. Given two masks $y_{true}, y_{pred}$ we evaluate 

$$IoU = \frac{y_{true} \cap y_{pred}}{y_{true} \cup y_{pred}}$$

In [0]:
def mean_iou(y_true, y_pred):
    yt0 = y_true[:,:,:,0]
    yp0 = K.cast(y_pred[:,:,:,0] > 0.5, 'float32')
    inter = tf.count_nonzero(tf.logical_and(tf.equal(yt0, 1), tf.equal(yp0, 1)))
    union = tf.count_nonzero(tf.add(yt0, yp0))
    iou = tf.where(tf.equal(union, 0), 1., tf.cast(inter/union, 'float32'))
    return iou

## U-Net Model

In [0]:
def unet(sz = (512, 512, 3)):
  x = Input(sz)
  inputs = x
  
  #down sampling 
  f = 8
  layers = []
  
  for i in range(0, 6):
    x = Conv2D(f, 3, activation='relu', padding='same') (x)
    x = Conv2D(f, 3, activation='relu', padding='same') (x)
    layers.append(x)
    x = MaxPooling2D() (x)
    f = f*2
  ff2 = 64 
  
  #bottleneck 
  j = len(layers) - 1
  x = Conv2D(f, 3, activation='relu', padding='same') (x)
  x = Conv2D(f, 3, activation='relu', padding='same') (x)
  x = Conv2DTranspose(ff2, 2, strides=(2, 2), padding='same') (x)
  x = Concatenate(axis=3)([x, layers[j]])
  j = j -1 
  
  #upsampling 
  for i in range(0, 5):
    ff2 = ff2//2
    f = f // 2 
    x = Conv2D(f, 3, activation='relu', padding='same') (x)
    x = Conv2D(f, 3, activation='relu', padding='same') (x)
    x = Conv2DTranspose(ff2, 2, strides=(2, 2), padding='same') (x)
    x = Concatenate(axis=3)([x, layers[j]])
    j = j -1 
    
  
  #classification 
  x = Conv2D(f, 3, activation='relu', padding='same') (x)
  x = Conv2D(f, 3, activation='relu', padding='same') (x)
  outputs = Conv2D(1, 1, activation='sigmoid') (x)
  
  #model creation 
  model = Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = [mean_iou])
  
  return model

In [0]:
model = unet()

## Callbacks

Simple functions to save the model at each epoch and show some predictions 

In [0]:
def build_callbacks():
        checkpointer = ModelCheckpoint(filepath='unet.h5', verbose=0, save_best_only=True, save_weights_only=True)
        callbacks = [checkpointer, PlotLearning()]
        return callbacks

# inheritance for training process plot 
class PlotLearning(keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        #self.fig = plt.figure()
        self.logs = []
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('mean_iou'))
        self.val_acc.append(logs.get('val_mean_iou'))
        self.i += 1
        print('i=',self.i,'loss=',logs.get('loss'),'val_loss=',logs.get('val_loss'),'mean_iou=',logs.get('mean_iou'),'val_mean_iou=',logs.get('val_mean_iou'))
        
        #choose a random test image and preprocess
        path = np.random.choice(test_files)
        raw = Image.open(f'images/{path}')
        raw = np.array(raw.resize((512, 512)))/255.
        raw = raw[:,:,0:3]
        
        #predict the mask 
        pred = model.predict(np.expand_dims(raw, 0))
        
        #mask post-processing 
        msk  = pred.squeeze()
        msk = np.stack((msk,)*3, axis=-1)
        msk[msk >= 0.5] = 1 
        msk[msk < 0.5] = 0 
        
        #show the mask and the segmented image 
        combined = np.concatenate([raw, msk, raw* msk], axis = 1)
        plt.axis('off')
        plt.imshow(combined)
        plt.show()

## Training

We're going to define the number of steps per epoch (based on the batch size) and also the number of epochs

In [0]:
train_steps = len(train_files) //batch_size
test_steps = len(test_files) //batch_size

print(train_steps)
print(test_steps)

In [0]:
history = model.fit_generator(train_generator, 
                    epochs = 100, steps_per_epoch = train_steps,validation_data = test_generator, validation_steps = test_steps,
                    callbacks = build_callbacks(), verbose = 0)

`history` contains the training and validation metrics that we can then plot as a function of epoch

In [0]:
print(history.history.keys())

In [0]:
# summarize history for iou
plt.plot(history.history['mean_iou'])
plt.plot(history.history['val_mean_iou'])
plt.title('model IoU')
plt.ylabel('IoU')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Testing

On an unseen image ...

In [0]:
raw = Image.open('test/buck-farm-vegetation-survey.jpg')
raw = np.array(raw.resize((512, 512)))/255.
raw = raw[:,:,0:3]

#predict the mask 
pred = model.predict(np.expand_dims(raw, 0))

#mask post-processing 
msk  = pred.squeeze()
msk = np.stack((msk,)*3, axis=-1)
msk[msk >= 0.5] = 1 
msk[msk < 0.5] = 0 

#show the mask and the segmented image 
combined = np.concatenate([raw, msk, raw* msk], axis = 1)
plt.axis('off')
plt.imshow(combined)
plt.show()

## References


1.   http://deeplearning.net/tutorial/unet.html
2.   https://github.com/ldenoue/keras-unet



CSDMS	2019	Annual	Meeting:	CSDMS	3.0	– Bridging	Boundaries

May	21-23,	2019

### Daniel Buscombe

Assistant Research Professor

School of Earth and Sustainability

School of Informatics, Computing and Cyber Systems


Northern Arizona University, Flagstaff, AZ

[Email](mailto:daniel.buscombe@nau.edu)
[Web](www.danielbuscombe.com)
[Google Scholar](https://scholar.google.com/citations?user=bwVl0NwAAAAJ&hl=en)